In [1]:
import pandas as pd

In [2]:
tab = pd.read_csv('file.csv', sep=';')
tab.reset_index(inplace=True)

# Обработка данных.

In [3]:
from lemmatization import prepare_lemmatized_text, remove_punctuation

In [4]:
tab['text_employer'] = remove_punctuation(tab)

In [5]:
tab['text'] = tab['text_employer'].apply(lambda x: prepare_lemmatized_text(x))

In [6]:
from stop_words import get_stop_words

In [7]:
stop_words = get_stop_words('ru')

In [8]:
from sklearn.feature_extraction.text import CountVectorizer

In [9]:
cv = CountVectorizer(stop_words=stop_words)

# Запуск LDA.

In [10]:
data_for_lda = cv.fit_transform(tab['text'])

In [11]:
from sklearn.decomposition import LatentDirichletAllocation as LDA

In [12]:
model = LDA(n_components=50, random_state=239)

In [13]:
model.fit(data_for_lda)

LatentDirichletAllocation(n_components=50, random_state=239)

In [14]:
import numpy as np

In [15]:
def get_keywords(text, cv, model, n_topics=3, n_keywords=5):
    vectorized = cv.transform([text])
    lda_topics = model.transform(vectorized)
    lda_topics = np.squeeze(lda_topics, axis=0)
    n_topics_indices = lda_topics.argsort()[-n_topics:][::-1]

    top_topics_words_dists = []
    for i in n_topics_indices:
        top_topics_words_dists.append(model.components_[i])

    shape=(n_keywords*n_topics, model.components_.shape[1])
    keywords = np.zeros(shape=shape)
    for i, topic in enumerate(top_topics_words_dists):
        n_keywords_indices = topic.argsort()[-n_keywords:][::-1]
        for k, j in enumerate(n_keywords_indices):
            keywords[i * n_keywords + k, j] = 1
    keywords = cv.inverse_transform(keywords)
    keywords = [keyword[0] for keyword in keywords]
    return keywords  

Мы берём первую тему и 3 ключевых слова, ей соответствующие в каждом случае.

Первая тема самая важная, одного слова может быть недостаточно.

In [16]:
res = []

for text in tab['text']:
    res.append(get_keywords(text, cv, model, 1, 3))

In [17]:
long_string = ' '.join([x[0] + ' ' +  x[1] + ' ' + x[2] for x in res])

In [18]:
topics = list(set(long_string.split(' ')))

In [19]:
len(topics)

97

In [20]:
topics

['ограничение',
 'физический_лицо',
 'эквайринг',
 'клиент',
 'терминал',
 'уточнить',
 'расчетный',
 'карта',
 'кабинет',
 'никак',
 'крестьянский',
 'извините',
 'пароль',
 'переводить',
 'банк',
 'надежда',
 'провести',
 'оператор',
 'город',
 'перевод',
 'наличные',
 'направить',
 'торговый_эквайринг',
 'изменение',
 'номер',
 'организация',
 'заявка',
 'сотрудник',
 'подпись',
 'поэтому',
 'лимит',
 'нужный',
 'информация',
 'оборот',
 'удобный',
 'сообщение',
 'закрыть',
 'кредит',
 'планировать',
 'здравствуйте',
 'коллега',
 'электронный',
 'услуга',
 'мобильный',
 'выбрать',
 'оплата',
 'хозяйство',
 'решение',
 'навигация',
 'просить',
 'смс',
 'отделение_банк',
 'подсказать',
 'акция',
 'выплата',
 'специалист',
 'понять',
 'подойти',
 'сын',
 'зарплатный_проект',
 'отделение',
 'добрый_день',
 'ноль',
 'торговый',
 'вопрос',
 'приложить',
 'доставка',
 'прямой',
 'перезвонить',
 'покупка',
 'рубль',
 'бизнес',
 'линия',
 'сайт',
 'индивидуальный',
 'офис_банка',
 'пакет_усл

Нужных нам тем немного. Проще всего сделать ручную разметку (см. Excel файл).

# Создание тем на основе ручной разметки.

In [21]:
direct_dict = {}

In [22]:
direct_dict["Приветствие"] = ["здравствуйте", "добрый_день"]

In [23]:
direct_dict["Перенос активности"] = ['оператор', 'поддержка', 'контакт', 'специалист', 'подойти', 'переводить', 'попробовать', 'коллега',
'отделение_банк', 'офис', 'звонить', 'линия', 'информация', 'сотрудник', 'отделение', 'направить', 'подсказать', 'поговорить', 'уточнить', 'адрес', 'банк', 'сайт', 'офис_банка', 'терминал']

In [24]:
direct_dict["Окончание разговора"] = ['прощение', 'просить', 'извините', 'перезвонить', 'никак']

In [25]:
direct_dict["Болтовня"] = ['надежда', 'ноль', 'сын', 'крестьянский', 'хозяйство', 'точка']

In [26]:
direct_dict["Следующие шаги"] = ['поэтому', 'провести', 'планировать', 'повод', 'приложить']

In [27]:
direct_dict["Безопасность"] = ['пароль', 'смс', 'логин', 'код', 'сообщение', 'подпись', 'лимит']

In [28]:
direct_dict["Тарифы или Цены"] = ['рубль', 'пакет_услуга']

In [29]:
direct_dict["Акции"] = ['услуга', 'комиссия', 'зарплатный_проект', 'индивидуальный', 'акция']

In [30]:
direct_dict["Удобство работы"] = ['удобный', 'доставка', 'кабинет', 'ограничение', 'электронный', 'мобильный']

In [31]:
direct_dict["Простота настройки"] = ["понять", "навигация", "нужный"]

In [32]:
direct_dict["Скорость работы"] = ["прямой"]

In [33]:
direct_dict["Преимущества"] = ['выбрать', 'закрыть', 'месяц', 'кредитование', 'кредит', 'заявка', 'бизнес']

In [34]:
direct_dict["Вовлечение или подведение к диалогу"] = ['наличные', 'платеж', 'клиент', 'деятельность', 'заинтересованность', 'физический_лицо', 'слышать', 'счет', 'организация']

In [35]:
direct_dict["Призыв к действию"] = ['решение', 'изменение', 'покупка', 'номер', 'оплата', 'перевод', 'город', 'расчетный', 'выплата', 'оборот']

In [36]:
direct_dict["Инструкция по подключению или приобретению"] = ["карта", "торговый_эквайринг", "эквайринг", "торговый"]

In [37]:
direct_dict["Вопрос клиента"] = ["вопрос"]

In [38]:
inversed_dict = {}
for key in direct_dict.keys():
    for value in direct_dict[key]:
        inversed_dict[value] = key

In [39]:
tab['topics'] = ""

In [40]:
for j in range(len(res)):
    words = res[j]
    analogs = list(set([inversed_dict[word] for word in words]))
    tab.loc[j, "topics"] = " ".join(analogs)

In [41]:
for key in direct_dict.keys():
    print(key)
    tabred = tab[tab['topics'].str.contains(key)]
    print(len(tabred))

Приветствие
69
Перенос активности
3476
Окончание разговора
1564
Болтовня
146
Следующие шаги
60
Безопасность
686
Тарифы или Цены
191
Акции
1404
Удобство работы
1446
Простота настройки
193
Скорость работы
6
Преимущества
1438
Вовлечение или подведение к диалогу
739
Призыв к действию
1490
Инструкция по подключению или приобретению
1556
Вопрос клиента
933


In [42]:
for key in direct_dict.keys():
    print(key)
    tabred = tab[tab['topics'].str.contains(key)]
    print(tabred['text'].tolist()[0])

Приветствие
добрый_день
Перенос активности
добрый_деть  это ваш персональный_менеджер_сбербанк бизнес звонить вы как индивидуальный_предприниматель вы удобный сейчас говорить  когда когда
Окончание разговора
добрый_деть  это ваш персональный_менеджер_сбербанк бизнес звонить вы как индивидуальный_предприниматель вы удобный сейчас говорить  когда когда
Болтовня
у я один один
Следующие шаги
выходной положить
Безопасность
добрый_день максим это персональный_менеджер_сбербанк блок сбер пизда звонить вы как индивидуальный_предприниматель вы удобный говорить  услуга который сбер бизнес рекомендовать для вы хотеть они рассказать чтобы ноль свой возможность  для вы сейчас доступный специальный предложение по бизнес карта карта позволять круглосуточно вносить снимать наличные с расчетный счет основный преимущество оплата ежедневный покупка снятие наличные плата комиссия_банк и фактически карта привязать к ваш расчетный_счет круглосуточный доступ и возможность подключить бизнес кэшбэк  просто сей

In [43]:
tab_new = pd.read_csv('file.csv', sep=';')
tab_new["topics"] = tab["topics"]

In [44]:
tab_new["topics"].to_csv("results_lda.csv")